In [1]:
import model_lib
import numpy as np
import warnings
warnings.filterwarnings('ignore', '.*output shape of zoom.*')
import pickle
import importlib
importlib.reload(model_lib)
import os
import time
from PIL import Image

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [2]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 64
    STEPS_PER_EPOCH = 100
    # not going to use these
    N_DISTORTIONS = 0
    MAX_DISTORTION = 0.3
    MIN_DISTORTION = -0.1
    
    VALIDATION_STEPS = 20
    # including gt
    NUM_CLASSES = 81
    # only flips
    IMAGE_AUGMENT = True

    MEAN_PIXEL = np.array([123.7, 116.8, 103.9],dtype=np.float32)
    MAX_GT_INSTANCES = 100
    DETECTION_MAX_INSTANCES = 100
    DETECTION_MIN_CONFIDENCE = 0.7
    CLASS_NAMES = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
        'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
        'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
        'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
        'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
        'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
    DETECTION_NMS_THRESHOLD = 0.3
    LEARNING_RATE = 0.05
    LEARNING_MOMENTUM = 0.9
    WEIGHT_DECAY = 0.0001
    WIDTH = 224
    HEIGHT = 224
    MASK_SHAPE = (64,64)
    GRID_WIDTH = 16
    GRID_HEIGHT = 16
    CLUE_SHAPE = (20,20)
    GRID_SHAPE = (GRID_WIDTH, GRID_HEIGHT)
    GRID_RESOLUTION = (1, 1)
    IS_PADDED = True
    MASK_THRESOLD = 0.7
    def __init__(self):
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT,3)
        self.MAX_BATCH_SIZE = self.BATCH_SIZE*32

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [3]:
root_dir = "/media/data/nishanth/aravind/"
config = ProposalConfig()
model_dir = "./models/"
train_dataset,val_dataset = None,None
train_pickle = root_dir+"val_cid.pickle"
val_pickle = root_dir+"val_cid.pickle"
class ClassInstancesDataset():
    def __init__(self):
        self.class_wise_instance_info = [[] for i in range(81)]
        self.instance_info = []
import time
start = time.time()
with open(train_pickle,"rb") as train_ann:
    train_cid = pickle.load(train_ann)
end = time.time()
print(end-start)
#     train_dataset = model_lib.CocoDataset(train_cid,config)
with open(val_pickle,"rb") as val_ann:
    val_cid = pickle.load(val_ann)
#     val_dataset = model_lib.CocoDataset(val_cid,config)
train_loader = model_lib.get_loader(train_cid,config,"ins")
val_loader = model_lib.get_loader(val_cid,config,"ins")

0.609081506729126


In [4]:
z = enumerate(train_loader)
import time
s = 0
N = 1000
for i in range(1,N):
    start = time.time()
    data = next(z)
    end = time.time()
    s += end-start
    print(end-start,s/i)
print(s/N)

0.5572962760925293  batch_collate
0.5886411666870117  batch_collate
0.4985029697418213  batch_collate
0.218580961227417  batch_collate
0.3834507465362549  batch_collate
0.37436676025390625  batch_collate
0.3261082172393799  batch_collate
0.38337135314941406  batch_collate
0.3203256130218506  batch_collate
10.291091918945312 10.291091918945312
0.46692442893981934  batch_collate
0.33815646171569824  batch_collate
0.4394052028656006  batch_collate
0.8688719272613525 5.5799819231033325
0.49556684494018555  batch_collate
0.4511990547180176  batch_collate
0.44844698905944824  batch_collate
0.6621518135070801  batch_collate
0.40909433364868164  batch_collate
0.44199204444885254 3.867318630218506
0.24059700965881348  batch_collate
0.19628524780273438  batch_collate
0.22516345977783203  batch_collate
0.2516777515411377  batch_collate
0.3268313407897949  batch_collate
0.5092175006866455  batch_collate
0.3013496398925781  batch_collate
0.20308947563171387  batch_collate
0.3153672218322754  batch_

0.3692631721496582  batch_collate
0.24700593948364258  batch_collate
0.08169078826904297  batch_collate
0.09187817573547363  batch_collate
0.36429929733276367  batch_collate
5.5508832931518555 0.4992289968056254
0.00016880035400390625 0.4943362497815899
0.022732019424438477 0.4897575679334622
0.00016379356384277344 0.48504993548760045
0.06460261344909668 0.48104567527770997
0.00020456314086914062 0.47650943837075865
0.04902362823486328 0.47251424388350727
0.0001742839813232422 0.4681407257362648
0.09843802452087402  batch_collate
0.07359480857849121 0.4645210384228908
0.00018167495727539062 0.46029977148229423
0.08619117736816406 0.45692942378757234
0.00020694732666015625 0.45285154453345705
0.0904238224029541 0.44964421955885087
0.00017023086547851562 0.4457014652720669
0.3643977642059326  batch_collate
0.2788212299346924 0.44425033279087234
0.00017786026000976562 0.44042212182077867
0.1049492359161377 0.4375548321976621
0.009912967681884766 0.43393074860007075
0.056094408035278320.25

0.17540287971496582  batch_collate
0.07106661796569824 0.4090155074946688
0.0001773834228515625 0.40708702577734895
0.05054283142089844 0.4054131093719196
0.019884109497070312 0.40361157198932684
0.05747413635253906 0.4020016304282255
0.00018715858459472656 0.4001413782437642
0.034932613372802734 0.39845838854389803
0.015735864639282227 0.3967027806360787
0.2533257007598877  batch_collate
0.09032225608825684  batch_collate
0.09299397468566895  batch_collate
0.041975975036621094 0.3950830235328848
0.0001938343048095703 0.3932880726727572
0.05965137481689453 0.39177840435666733
0.0002384185791015625 0.3900147107270387
0.19052839279174805 0.38912015324750826
0.0002353191375732422 0.38738406023808886
0.05970478057861328 0.3859277078840468
0.0022521018981933594 0.384230028211543
0.05446338653564453 0.38277731172838925
0.00019121170043945312 0.3810993025177403


IndexError: Traceback (most recent call last):
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/aravind/anaconda3/envs/myenv/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/aravind/re/model_lib.py", line 183, in __getitem__
    image, masks, class_id = self.load_image_gt(instance_index)
  File "/home/aravind/re/model_lib.py", line 270, in load_image_gt
    instance_info = dataset.instance_info[instance_index]
IndexError: list index out of range


0.1862649917602539  batch_collate
0.27295446395874023  batch_collate
0.11836528778076172  batch_collate
0.08787083625793457  batch_collate
0.38382959365844727  batch_collate
0.10935354232788086  batch_collate
0.19980144500732422  batch_collate
0.17328786849975586  batch_collate
0.4976942539215088  batch_collate
0.18766283988952637  batch_collate
0.09575080871582031  batch_collate
0.10171103477478027  batch_collate
0.0992424488067627  batch_collate
0.10544538497924805  batch_collate
0.13710665702819824  batch_collate
0.09762740135192871  batch_collate
0.10054922103881836  batch_collate
0.13123846054077148  batch_collate
0.0937960147857666  batch_collate
0.21152949333190918  batch_collate
0.20542216300964355  batch_collate
0.2550990581512451  batch_collate
0.2367231845855713  batch_collate
0.3207252025604248  batch_collate
0.20450663566589355  batch_collate
0.0823066234588623  batch_collate
0.25186848640441895  batch_collate
0.23706340789794922  batch_collate
0.20157647132873535  batch_c

In [ ]:
import torch
import torch.nn.functional as F
net = model_lib.SimpleHGModel()
net.load_state_dict(torch.load(model_dir+"model_bce_0_7600.pt"))
net = net.cuda()
with torch.no_grad():
    for i,data in enumerate(val_loader):
        batch_images,batch_impulses,batch_gt_responses,batch_class_ids = data
        a,b,c= batch_images.numpy(), batch_impulses.numpy(),batch_gt_responses.numpy()
        a = np.moveaxis(a,1,-1)
        b = np.moveaxis(b,1,-1)
        c = np.moveaxis(c,1,-1)
        Image.fromarray((a[0]*128 + config.MEAN_PIXEL).astype(np.uint8),"RGB").show()
        Image.fromarray((b[0][:,:,0]*128).astype(np.uint8),"L").show()
        Image.fromarray((c[0][:,:,0]*128).astype(np.uint8),"L").show()
        batch_images,batch_impulses,batch_gt_responses,batch_class_ids = batch_images.cuda(),batch_impulses.cuda(),batch_gt_responses.cuda(),batch_class_ids.cuda()
        pred_class = net([batch_images,batch_impulses])
#         print(pred_class)
        pred_class = F.softmax(pred_class,dim=-1).squeeze()
        maxs, indices = torch.topk(pred_class,5,-1)
        print(maxs.shape,indices.shape)
        for i in range(5):
            print(maxs[i],indices[i],config.CLASS_NAMES[int(indices[i])])
#         print(batch_class_ids)
#         print(indices)
#         print("gt_class:",config.CLASS_NAMES[int(batch_class_ids[0])])
#         print("pred_class: ",pred_class[int(batch_class_ids[0])])
#         print(config.CLASS_NAMES[int(indices[0])])
        input()

In [ ]:
import random
import time
np.set_printoptions(threshold=np.nan)
counts = [0 for i in range(81)]
train_loader = iter(train_loader)
for i in range(1000):
    start = time.time()
    g = next(train_loader)
    print(time.time()-start)
for i,g in enumerate(train_loader):
    print("sample:" + str(i))
#     start = time.time()
    batch_images,batch_impulses,batch_gt_responses,batch_class_ids = g
#     print(batch_class_ids)
    for i in batch_class_ids:
        counts[int(i.item())] += 1
print(counts)
#     batch_images = np.moveaxis(batch_images.numpy(),1,-1)
#     print(batch_images.shape)
#     batch_impulses = batch_impulses.squeeze().numpy()*128
#     batch_gt_responses = batch_gt_responses.squeeze().numpy()*128
#     batch_class_ids = batch_class_ids.numpy()
#     # s =(time.time()-start)
#     # print(s)
#     batch_images[0] *= 128
#     batch_images[0] += config.MEAN_PIXEL
#     img = Image.fromarray((batch_images[0]).astype("uint8"),"RGB")
#     img.show()
#     print((np.sum(batch_gt_responses)//128)**0.5)
#     mask = Image.fromarray(((batch_gt_responses)).astype("uint8"),"L")
#     mask.show()
#     print(config.CLASS_NAMES[batch_class_ids[0]])
#     impulse = Image.fromarray(((batch_impulses)).astype("uint8"),"L")
#     impulse.show()
#     input()

In [ ]:
[len(c) for c in train_cid.class_wise_instance_info]